In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [7]:
print(text[300:550])

 kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority sur


In [9]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters, {vocab}")

65 unique characters, ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Process the Text

In [11]:
char2idx = {
    u: i for i, u in enumerate(vocab)
}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [12]:
ziped = zip(char2idx, range(20))

In [16]:
print(f"{repr(text[:13])} ---- characters mapped to int ---> {text_as_int[:13]}")

'First Citizen' ---- characters mapped to int ---> [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Create training and target samples

In [17]:
seq_length = 100
examples_per_epoch = len(text)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [21]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [23]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [39]:
for input_example, output_example in dataset.take(1):
    print(f"Input: {repr(''.join(idx2char[input_example.numpy()]))}")
    print(f"Output: {repr(''.join(idx2char[output_example]))}")
    print(" ")

Input: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Output: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
 


In [40]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], output_example[:5])):
    print(f"Step {i}")
    print(f" Input: {input_idx} {repr(idx2char[input_idx])}")
    print(f" Expected: {target_idx} ({repr(idx2char[target_idx])})")

Step 0
 Input: 18 'F'
 Expected: 47 ('i')
Step 1
 Input: 47 'i'
 Expected: 56 ('r')
Step 2
 Input: 56 'r'
 Expected: 57 ('s')
Step 3
 Input: 57 's'
 Expected: 58 ('t')
Step 4
 Input: 58 't'
 Expected: 1 (' ')


### Create trianing batches

In [41]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [42]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [50]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size, embedding_dim, batch_input_shape=[batch_size, None]
        ),
        tf.keras.layers.GRU(
            rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'
        ),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model



In [51]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [52]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_3 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [54]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

In [55]:
sampled_indices

array([ 0, 49, 60, 41, 63, 54, 30, 16, 40, 47, 51,  7, 42, 52, 47,  7, 16,
       48, 23, 37, 48, 22, 55, 55, 23, 38, 38, 63, 37, 26, 24, 39, 34, 12,
        2, 29, 60,  7, 24, 52, 54, 11, 22, 18, 64, 49, 22, 50, 46, 61,  4,
       29, 58, 12, 27, 63, 41, 25, 33, 26, 13, 64, 55, 24, 19, 53, 60,  9,
       15, 53, 14, 32, 29,  4, 32, 35, 29, 53, 51, 41, 49, 34, 21,  2, 24,
       32, 44, 16, 61,  9, 48, 33, 20, 37, 32, 27, 42,  1, 32, 39])

In [58]:
print(f"Input: \n", {repr("".join(idx2char[input_example_batch[0]]))})
print()
print(f"Next Char Predictions: \n", {repr("".join(idx2char[sampled_indices]))})

Input: 
 {'":\\nNor I, nor any\\nBut one that\'s here, and that\'s himself, for he\\nThe sacred honour of himself, his q"'}

Next Char Predictions: 
 {"'\\nkvcypRDbim-dni-DjKYjJqqKZZyYNLaV?!Qv-Lnp;JFzkJlhw&Qt?OycMUNAzqLGov3CoBTQ&TWQomckVI!LTfDw3jUHYTOd Ta'"}


In [59]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print(f"Prediction Shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print(f"Scalar Loss: ", example_batch_loss.numpy().mean())

Prediction Shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
Scalar Loss:  4.174137


In [60]:
model.compile(optimizer="adam", loss=loss)

In [61]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [62]:
EPOCHS=10

In [63]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 359s 2s/step - loss: 2.6675
Epoch 2/10
172/172 [==============================] - 555s 3s/step - loss: 1.9672
Epoch 3/10
172/172 [==============================] - 641s 4s/step - loss: 1.6972
Epoch 4/10
172/172 [==============================] - 615s 4s/step - loss: 1.5459
Epoch 5/10
172/172 [==============================] - 550s 3s/step - loss: 1.4560
Epoch 6/10
172/172 [==============================] - 599s 3s/step - loss: 1.3950
Epoch 7/10
172/172 [==============================] - 586s 3s/step - loss: 1.3494
Epoch 8/10
172/172 [==============================] - 551s 3s/step - loss: 1.3098
Epoch 9/10
172/172 [==============================] - 525s 3s/step - loss: 1.2760
Epoch 10/10
172/172 [==============================] - 630s 4s/step - loss: 1.2439


## Generate Text

In [65]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [66]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [67]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [70]:
def generate_text(model, start_string):
    
    num_generate = 1000
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return (start_string) + ''.join(text_generated)

In [71]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: tell me with a vouch it.
Hele, that I, my master cancle and be a dismat all.
The people
Where he s wrong which was confesson
But from her.

CORIOLANUS:
Sir this and the posthy turn, and set you to your resobution.

YORK:
Save Plaulest the maid when it?

Clongeto to shrigk?

QUEEN ELIZABETH:
Mounts the Clare-transposide well me with it!

GRUMIO:
O Thou, presust now; 'twose that say
chose that weinou, as very hand make thee but her much in kind my brother
By night with danners from Sorrow;
ere that my good lord;
Ithink it is a chint; wherein our strongest hand,
The lands in prisoner of my lord.

KATHARINA:
Come, then?

MYRCUT:
An if I give you his new cale and Roman:
O, what is your duest that we will to complament the quarrels
To grief of wanton presumbt: the formad still the devile,
Can he fresh with what of entertain but shirs at Franes. Fare you, leave to be tradicate.
With warld.

QUEEN MARGARET:
Here is me, as this an oice gates: father,
Behold my nusses; not touch thy hand 

## Custom Training, Gradient Taping

In [72]:
custom_model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)
optimizer = tf.keras.optimizers.Adam()

In [73]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = custom_model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True
            )
        )
    grads = tape.gradient(loss, custom_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, custom_model.trainable_variables))
    
    return loss

In [75]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    hidden = custom_model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if(batch_n % 100 == 0):
            template = f' Epoch {epoch + 1} Batch {batch_n} Loss {loss}'
            
    if(epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print(f'Epoch {epoch + 1} Loss {loss}')
    print(f'Time taken for 1 epoch {time.time() - start} sec\n')

Epoch 1 Loss 1.7925949096679688
Time taken for 1 epoch 289.62099599838257 sec

Epoch 2 Loss 1.5844006538391113
Time taken for 1 epoch 268.42560625076294 sec

Epoch 3 Loss 1.4657706022262573
Time taken for 1 epoch 8550.964779138565 sec

Epoch 4 Loss 1.4447128772735596
Time taken for 1 epoch 295.9449589252472 sec

Epoch 5 Loss 1.3334530591964722
Time taken for 1 epoch 436.3803300857544 sec

Epoch 6 Loss 1.3238227367401123
Time taken for 1 epoch 450.3049740791321 sec

Epoch 7 Loss 1.2752586603164673
Time taken for 1 epoch 477.0326919555664 sec

Epoch 8 Loss 1.253023386001587
Time taken for 1 epoch 461.5972020626068 sec

Epoch 9 Loss 1.263017177581787
Time taken for 1 epoch 414.66977310180664 sec

Epoch 10 Loss 1.2234126329421997
Time taken for 1 epoch 426.545058965683 sec

